In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

import warnings
warnings.filterwarnings('ignore')

In [2]:
# prep info
# contact: c
# status inlcude: r, s, a

root_path = r'/Users/xiaoqiansun/Desktop/MedLu/TubeTest/Data'


days = 6
fpsN = 15
fpsB = 30
seconds = 5
intervalLength = 20
frames = seconds*fpsN
baselinePeriod = 180 #(3min(180s) baseline)
redundentTime = 190 #10s+3min


M32_list = ['32-1_28', '32-2_18', '32-3_15', '32-4_20', '32-5_12', '32-6_15']
M33_list = ['33-1_28', '33-2_35', '33-3_32', '33-4_30', '33-5_30', '33-6_26']



# Define

In [3]:
# to keep certain status frrame interval
#----------------------------------------------------------------------------------
def pick_ARInterval_ON(df, which_mouse, behavior, folderPath):      
      
    df1 = df.copy().reset_index(drop=True)
    dfb = df1[df1[which_mouse]== behavior]
    df1 = df1.drop(['mouse1', 'mouse2', 'contact'], axis=1)
    dfb = dfb.drop(['mouse1', 'mouse2', 'contact'], axis=1)
    
    
    # read in ON/OFF summary dataframe
    file_name = 'ONOFF.csv'
    on_off_df = pd.read_csv(os.path.join(folderPath, 'ONOFF.csv'))
    columns = on_off_df.columns.to_list()
    columns[0] = 'Neurons'
    on_off_df.columns = columns
    on_off_df = on_off_df.set_index('Neurons')
    # get approach/retreat on neurons
    if behavior == 'a':
        ON_Neurons = on_off_df[on_off_df['approachON'] == 1].index.to_list()
    elif behavior == 'r':
        ON_Neurons = on_off_df[on_off_df['retreatON'] == 1].index.to_list()
        
    
    #loc
    if len(ON_Neurons) == 0:
        return (pd.DataFrame(), pd.DataFrame())
    else:
        df1 = df1[ON_Neurons]
        dfb = dfb[ON_Neurons]
        return (df1, dfb)
        

In [4]:
# get social interval list
#----------------------------------------------------------------------------------
def get_BonsetA(df, df_b, fps, seconds, intervalLength):
    
    # get behavior index
    index_1_list = df_b.index.tolist()
    
    # get intervals list
    if len(index_1_list) >= seconds*fps:
        # get intervals list(>seconds*fps)
        interval_list = []
        interval_list.append(index_1_list[0])

        for i in range(len(index_1_list)):
            if i < len(index_1_list)-1:
                if index_1_list[i+1] - index_1_list[i] > intervalLength:
                    interval_list.append(index_1_list[i])
                    interval_list.append(index_1_list[i+1])

        interval_list.append(index_1_list[-1])

        # split interval list into sublist
        composite_interval_list = [interval_list[x:x+2] for x in range(0, len(interval_list),2)]

        
        df_Bonset_list = []
        df_onsetA_list = []
        # enlargy each interval 2# frames (# forward, # afterward)
        for interval in composite_interval_list:
            if interval[0] > seconds*fps:
                Bonset_start = interval[0] - seconds*fps
                Bonset_end = interval[0]
                onsetA_start = interval[0]
                if interval[0] + seconds*fps < len(df):
                    onsetA_end = interval[0] + seconds*fps 
                    #----------------locate----------------
                    Bonset_df = df.iloc[Bonset_start:Bonset_end]
                    df_Bonset_list.append(Bonset_df)

                    onsetA_df = df.iloc[onsetA_start:onsetA_end]
                    df_onsetA_list.append(onsetA_df)
            
            
    else: 
        df_Bonset_list = []
        df_onsetA_list = []
    
    return(df_Bonset_list, df_onsetA_list)



In [5]:
# calculate mean of each mean df from a df list
def calculate_avg_FF(df_list, frames):
    
    Mean_DF = pd.DataFrame()
    
    if len(df_list) > 0:
        for i in range(len(df_list)):
            if df_list[i].shape[0] == frames:
                Mean_DF['mean'+str(i)] = df_list[i].mean(axis=1).values.tolist()
        Mean_DF['MEAN'] = Mean_DF.mean(axis=1)
    
    return(Mean_DF)



def combine_BA(B_list, A_list, mouse_list, diffDF_list, frames, mouse_name, day):
    
    if  len(B_list) > 0:
        B_Mean = calculate_avg_FF(B_list, frames)
        A_Mean = calculate_avg_FF(A_list, frames)

        df_FF_Diff = {'mouse_name': mouse_name,
                      'day':day,
                      'Max-Max': max(A_Mean['MEAN'])-max(B_Mean['MEAN']),
                      'Mean-Mean': np.mean(A_Mean['MEAN'])-np.mean(B_Mean['MEAN'])}
        dic_FF_Diff = pd.DataFrame.from_dict(df_FF_Diff, orient='index').T
        diffDF_list.append(dic_FF_Diff)
 
        mouse_list.append( pd.DataFrame({'day'+str(day):B_Mean['MEAN'].values.tolist()+A_Mean['MEAN'].values.tolist()} ))
    
    return(mouse_list, diffDF_list)



In [6]:
def generate_meanSEM_Summary(df_list):
    
    DF = pd.concat(df_list, axis=1)
    
    summary = pd.DataFrame()
    summary['ALL_MICE_MEAN'] = DF.mean(axis = 1)
    summary['MEAN_SEM'] = DF.mean(axis = 1) - DF.sem(axis = 1) 
    summary['MEANSEM'] = DF.mean(axis = 1) + DF.sem(axis = 1) 
    summary['SEM'] = DF.sem(axis = 1)
    summary['MAX'] = DF.max(axis = 1)
    summary['MIN'] = DF.min(axis = 1)
        
    return(summary)


# Get Avg FF

- approach ON around approach

In [7]:
m32Diff_list = []
m33Diff_list = []
m32_approachS = []
m33_approachS = []


for day in range(days):

    # read in dataframe
    m32_folder = os.path.join(root_path, M32_list[day])
    m33_folder = os.path.join(root_path, M33_list[day])

    # read in dataframe
    m32_tube = pd.read_csv(os.path.join(m32_folder,'m32_tube.csv'), index_col = 'Frame')
    m33_tube = pd.read_csv(os.path.join(m33_folder,'m33_tube.csv'), index_col = 'Frame')   

    # filter on neuorns
    m32_aAP, m32_AP = pick_ARInterval_ON(m32_tube, 'mouse1', 'a', m32_folder)
    m33_aAP, m33_AP = pick_ARInterval_ON(m33_tube, 'mouse2', 'a', m33_folder)
    
    if len(m32_AP) != 0:
        # separate onset intervals; update onset data & diff info
        m32_BAP, m32_APA = get_BonsetA(m32_aAP, m32_AP, fpsN, seconds, intervalLength)
        m32_approachS, m32Diff_list = combine_BA(m32_BAP, m32_APA, m32_approachS, m32Diff_list, frames, 'm32', day+1)
    if len(m33_AP) != 0:
        m33_BAP, m33_APA = get_BonsetA(m33_aAP, m33_AP, fpsN, seconds, intervalLength)
        m33_approachS, m33Diff_list = combine_BA(m33_BAP, m33_APA, m33_approachS, m33Diff_list, frames, 'm33', day+1)


FF32_Diff = pd.concat(m32Diff_list)
FF32_Diff.to_excel('mouse32_'+str(seconds)+'_diff.xlsx')
FF33_Diff = pd.concat(m33Diff_list)
FF33_Diff.to_excel('mouse33_'+str(seconds)+'_diff.xlsx')

BapproaceA_m32_SummaryS = generate_meanSEM_Summary(m32_approachS)
BapproaceA_m33_SummaryS = generate_meanSEM_Summary(m33_approachS)


### plot Trace

In [8]:
def plot_MeanSEM_fill(sum1, sum2, seconds, behavior, smooth=False):
    
    # get data
    #--------------------------------------------------------
    ySum1 =  sum1['ALL_MICE_MEAN'].values.tolist()
    ySum1 = savgol_filter(ySum1, 35, 3)
    ySum1_MM_SEM = sum1['MEAN_SEM'].values.tolist()
    ySum1_MMSEM = sum1['MEANSEM'].values.tolist()
    
    ySum2 =  sum2['ALL_MICE_MEAN'].values.tolist()
    ySum2_MM_SEM = sum2['MEAN_SEM'].values.tolist()
    ySum2_MMSEM = sum2['MEANSEM'].values.tolist()
    
    x = sum1.index.values.tolist()
    
    
    if smooth:
        ySum1 = savgol_filter(ySum1, 35, 3)
        ySum1_MM_SEM = savgol_filter(ySum1_MM_SEM, 35, 3)
        ySum1_MMSEM = savgol_filter(ySum1_MMSEM, 35, 3)
        ySum2 = savgol_filter(ySum2, 35, 3)
        ySum2_MM_SEM = savgol_filter(ySum2_MM_SEM, 35, 3)
        ySum2_MMSEM = savgol_filter(ySum2_MMSEM, 35, 3)
        
    #ymin = min(min(ySum1), min(ySum2), min(ySum1_MM_SEM), min(ySum2_MM_SEM))-0.1
    #ymax = max(max(ySum1), max(ySum2), max(ySum1_MMSEM), max(ySum2_MMSEM))+0.1
    ymin = min(min(ySum1), min(ySum2))-0.1
    ymax = max(max(ySum1), max(ySum2))+0.1
        
    
    # plot
    #--------------------------------------------------------
    plt.figure(figsize = (8,4))

    plt.plot(x, ySum1, color = '#4f4f4f', lw=2, label='mouse32') ##black-WT
    plt.plot(x, ySum2, color = '#FF7C80', lw=2, label='mouse33')
    plt.plot([x[int(len(x)/2)],x[int(len(x)/2)]],[ymin+0.05,ymax-0.05] , '--r', lw = 2)

    # fill
    plt.fill_between(x, ySum1_MM_SEM, ySum1_MMSEM, color = '#dddddd', alpha = 0.5)
    plt.fill_between(x, ySum2_MM_SEM, ySum2_MMSEM, color = '#ffd9da', alpha = 0.5)

    plt.legend()
    plt.ylim(ymin, ymax)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().axes.get_xaxis().set_visible(False)

    title = behavior+'_'+str(seconds)+'s_AVG_FF'
    plt.savefig(title)
    
    plt.close()

In [10]:
# call
plot_MeanSEM_fill(BapproaceA_m32_SummaryS, BapproaceA_m33_SummaryS, seconds, 
                  #'smoothed_approachONapproach', smooth=True
                  'approachONapproach', smooth=False
                 )
                 
                  
